In [2]:
import pandas as pd
import numpy as np
import glob
import docx
import openpyxl
import time
import os
from stanfordnlp.server import CoreNLPClient
from sklearn.feature_extraction.text import TfidfVectorizer
from operator import itemgetter
from string import digits
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize

In [3]:
def SentimentScore(paragraph, weight=1):
    sentences = []
    lines_list = tokenize.sent_tokenize(paragraph)
    sentences.extend(lines_list)
    sid = SentimentIntensityAnalyzer()
    sentiment = 0
    for sentence in sentences:
        ss = sid.polarity_scores(sentence)
        score = ss['compound']
        if score < 0:
            sentiment += score*weight
        else:
            sentiment += score
    return round(sentiment/len(sentences),2)

def neg(bucket):
    if bucket < 4:
        return bucket + 4
    else:
        return bucket - 4
    
def remove_nan(l):
    return [x for x in l if not pd.isnull(x)]

def get_text(doc_names):
    documents = []
    titles = []
    remove_digits = str.maketrans('', '', digits)
    for i in range(len(doc_names)):
        doc_name = doc_names[i]
        paragraphs = docx.Document(doc_name).paragraphs
        par_text = [paragraph.text.replace('\xa0', ' ') for paragraph in paragraphs]
        text = par_text[0]
        titles.append(text)
        for i in range(1, len(par_text)):
            par = par_text[i]
            if par:
                text += ' ' + par
        documents.append(text.lower().translate(remove_digits))
    return titles, documents

def bin_of_words(filename):
    df = pd.read_excel(filename)
    return [sorted(list(set(remove_nan(df[col].values)))) for col in df.columns]

In [5]:
CoreNLP = "/home/roguehydra/Documents/Jaar 3/TOFU/SA/stanford-corenlp"
os.environ["CORENLP_HOME"] = CoreNLP
doc_names = sorted(glob.glob('/home/roguehydra/Documents/Jaar 4/TOFU/SA/Webpages/TEST/*.doc*'))
titles, documents = get_text(doc_names)
print("Amount of docs: {}\n".format(len(documents)))
begin = time.time()
ann_doc = []
# set up the client
with CoreNLPClient(annotators=['tokenize','ssplit','pos','depparse'], timeout=60000, memory='16G') as client:
    # annotate documents on the server
    for text in documents:
        ann_doc.append(client.annotate(text))
        
negations = []
for ann in ann_doc:
    negated=[]
    token_dict = {}
    for i in range(len(ann.sentence)):
        # get the sentence
        sentence = ann.sentence[i]

        # get the dependency parse of the sentence
        dependency_parse = sentence.basicDependencies

        #get a dictionary associating each token/node with its label
        for j in range(0, len(sentence.token)) :
            token_dict[sentence.token[j].tokenEndIndex] = sentence.token[j].word

        #take previous sentences into consideration
        offset = sentence.token[0].tokenBeginIndex        

        #get a list of the dependencies with the words they connect
        for item in dependency_parse.edge:
            dep = item.dep
            if dep == 'neg':
                source_node = item.source
                source_name = token_dict[offset + source_node]
                negated.append(source_name)
    negations.append(negated)
end = time.time()
print("\nElapsed time: {}s".format(round(end - begin,1)))

Amount of docs: 10

Starting server with command: java -Xmx16G -cp /home/roguehydra/Documents/Jaar 3/TOFU/SA/stanford-corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-4b0c8d716ea84b9c.props -preload tokenize,ssplit,pos,depparse


PermanentlyFailedException: Timed out waiting for service to come alive.

1: Dream 

2: Economic

3: Health

4: Environment

5: Nightmare

In [6]:
bins_of_words = bin_of_words('BINS - version 2.xlsx')

In [7]:
word_matrix = np.empty((len(documents), len(bins_of_words)), dtype=object)

In [8]:
for i in range(len(documents)):
    document = documents[i]
    for j in range(len(bins_of_words)):
        if str(word_matrix[i][j]) == 'None':
            word_matrix[i][j] = []
        bucket = bins_of_words[j]
        for word in bucket:
            if word in document:
                for _ in range(document.count(word)):
                    tmp = word_matrix[i][j].copy()
                    tmp.append(word)
                    word_matrix[i][j] = tmp

In [ ]:
rows = [0,4]
for i in range(len(negations)):
    negated = negations[i]
    for word in negated:
        
        for j in rows:
            
            if word in word_matrix[i][j]:
                tmp = word_matrix[i][j].copy()
                tmp.pop(tmp.index(word))
                word_matrix[i][j] = tmp
                
                j2 = neg(j)
                tmp2 = word_matrix[i][j2].copy()
                tmp2.append(word)
                word_matrix[i][j2] = tmp2 
                break


In [9]:
score_matrix = np.zeros((len(documents), len(bins_of_words)))

In [10]:
for i in range(len(documents)):
    for j in range(len(bins_of_words)):
        score_matrix[i][j] = len(word_matrix[i][j])

In [11]:
for i in range(len(documents)):
    print("Document {} \n\n{}\n".format(i+1, titles[i]))
    doc = documents[i]
    vader_score = SentimentScore(doc,2)
    positive = score_matrix[i][0]
    negative = score_matrix[i][4]
    total_sentiment = positive - negative
    
    economic = score_matrix[i][1] 
    health = score_matrix[i][2] 
    environment = score_matrix[i][3]
    total_topic =  economic + health + environment
    if total_sentiment > 0:
        sen_score = total_sentiment/positive
    elif total_sentiment < 0:
        sen_score = total_sentiment/negative
    elif total_sentiment == 0:
        sen_score = 0
        
    score = (sen_score + vader_score) / 2
    if score > 0.1:
        judgement = 'positive'
    elif score < -0.1:
        judgement = 'negative'
    elif score == 0:
        judgement = 'neutral'
    elif score > 0:
        judgement = 'neutral (leaning towards positive)'
    else:
        judgement = 'neutral (leaning towards negative)'
    
    
    print("Sentiment (score) : {}, ({})\n".format(judgement,round(score,2)))

    print("Economic score    : {}".format(round(economic/total_topic,2)))
    print("Health score      : {}".format(round(health/total_topic,2)))
    print("Evironmental score: {}".format(round(environment/total_topic,2)))
    print("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - ")

Document 1 

Geothermal Basics

Sentiment (score) : positive, (0.42)

Economic score    : 0.36
Health score      : 0.11
Evironmental score: 0.53
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Document 2 

Geothermal Energy Information and Facts

Sentiment (score) : neutral (leaning towards positive), (0.09)

Economic score    : 0.32
Health score      : 0.12
Evironmental score: 0.56
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Document 3 

Heat Without Fire: Geothermal For A Cleaner, Sustainable Future In New York City

Sentiment (score) : positive, (0.48)

Economic score    : 0.27
Health score      : 0.19
Evironmental score: 0.54
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Document 4 

Geothermal Heating and Cooling Technologies

Sentiment (score) : positive, (0.3)

Economic score    : 0.32
Health score      : 0.18
Evironmental score: 0.5
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Document 5 

Geothermal for Canad

In [ ]:
vectorizer = TfidfVectorizer(use_idf=True, stop_words='english', ngram_range=(1,3), min_df=2)
x = vectorizer.fit_transform(documents)
tf_idf_scores = x.toarray()
words = vectorizer.get_feature_names()
all_words = [n for m in bins_of_words for n in m]

In [ ]:
for i in range(len(tf_idf_scores)):
    print('Document {}\n'.format(i+1))
    document = tf_idf_scores[i]
    sug = []
    for j in range(len(document)):
        score = document[j]
        if score > 0:
            word = words[j]
            if word not in all_words:
                sug.append((score, word))

    if sug != []:
        sug.sort(key=itemgetter(0), reverse=True)
        for s, word in sug[-100:-90]:
            print(word)
    print('- - - - - - - - - - - - - - - -\n')